#Quiz
> tmdb_5000_movies에서 overview를 이용한 추천 시스템을 구축하시오.
* 코드 완성 후 [Quiz_이름.ipynb]로 제출

#1. 파일 읽기
> tmdb_5000_movies.csv 파일을 읽어 크기를 확인하세요

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

path = '/content/drive/MyDrive/1. 데이터 분석을 위한 기초 통계 수학/5. 컨텐츠 기반/data/tmdb_5000_movies.csv'
df = pd.read_csv(path)
df.shape

(4803, 20)

> 4803개의 영화와 20개의 feature가 존재

#2. feature 줄이기

In [3]:
df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


> 위의 feature 중 다음과 같은 내용만 추출해 보자
* id : 영화 아이디
* title : 영화 제목
* genres : 장르
* vote_average : 평점
* vote_count : 투표수
* popularity : 인기도
* keywords : 주요 키워드
* overview : 영화 개요

In [4]:
df_movies = df[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
df_movies

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",6.6,238,14.269792,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,72766,Newlyweds,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",5.9,5,0.642552,[],A newlywed couple's honeymoon is upended by th...
4800,231617,"Signed, Sealed, Delivered","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",7.0,6,1.444476,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],When ambitious New York attorney Sam is sent t...


##3. 데이터 클린징
> 위의 데이터를 이용하여 이상치 및 결측치를 확인하세요

###3.1 결측치 확인

In [5]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


> 전체 4803개의 데이터 중 overview 결측치가 3개

####3.1.1 결측치 처리

In [ ]:
df_movies = df_movies.dropna(subset=['overview'])
df_movies.info()

> overview의 키워드를 연관으로 만들것이라
overview에 결측치가 포함되어 있는 데이터는 필요 없으므로 열 결측치 제거 

###3.2 이상치 확인

In [24]:
df_movies.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.800000e+03,4800.000000,4800.000000,4.800000e+03,4800.000000,4800.000000,4800.000000
mean,2.905988e+07,56967.252917,21.505403,8.231205e+07,106.880833,6.092917,690.645208
std,4.073043e+07,88350.548128,31.822273,1.628950e+08,22.611663,1.191468,1234.853376
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.950000e+05,9012.750000,4.682212,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14623.500000,12.928897,1.918199e+07,103.000000,6.200000,236.000000
75%,4.000000e+07,58512.500000,28.350628,9.293886e+07,118.000000,6.800000,737.250000
max,3.800000e+08,447027.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


#4. 데이터 가공

In [29]:
import pandas as pd
pd.set_option('max_colwidth', 300)
df_movies['overview'][:3]

0                                                                     In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.
1                                                                    Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.
2    A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.
Name: overview, dtype: object

In [30]:
overview_data = 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'
print(type(overview_data))

<class 'str'>


#5. feature 생성

In [59]:
df_movies['overview_literal'] = df_movies['overview'].apply(lambda x : ('').join(x))
df_movies['overview_literal'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(ngram_range=(1, 2)).fit(df_movies['overview_literal'])
count_vect.vocabulary_

In [ ]:
count_vect = CountVectorizer(min_df=0, ngram_range=(0, 1)).fit(df_movies['overview_literal'])
count_vect.vocabulary_

In [81]:
overview_matrix = count_vect.fit_transform(df_movies['overview_literal'])
overview_matrix.shape

(4800, 21262)

> overview를 총 21262개의 feature로 구분하였으나 feature의 개수가 너무 많아
  in, the 등의 키워드와 관련없는 feature들을 필터링

#6. 유사도 분석

In [82]:
from sklearn.metrics.pairwise import cosine_similarity

overview_similarity = cosine_similarity(overview_matrix, overview_matrix)
overview_similarity[:3]

array([[1.        , 0.33436693, 0.26462806, ..., 0.32142857, 0.22681618,
        0.19015971],
       [0.33436693, 1.        , 0.35098206, ..., 0.32043497, 0.33783196,
        0.21429865],
       [0.26462806, 0.35098206, 1.        , ..., 0.33078508, 0.22917462,
        0.2191785 ]])

In [83]:
overview_similarity_sorted_idx = overview_similarity.argsort()[:,::-1]
overview_similarity_sorted_idx[:3]

array([[   0, 3603, 3191, ..., 4510, 3487, 4631],
       [   1,  137,  274, ..., 3608, 4510,  161],
       [   2, 3932,  432, ..., 1111, 1770, 3492]])

> 0번 영화와 가장 유사한 영화는 3603 -> 3191 순인 것을 알 수 있다. 

In [84]:
print(df_movies.iloc[0]['title'])
print(df_movies.iloc[3603]['title'])
print(df_movies.iloc[3191]['title'])
print('==============================')
df_movies.iloc[0]

Avatar
Apollo 18
Two Evil Eyes


budget                                                                                                                                                                                                                                                                                                                    237000000
genres                                                                                                                                                                                       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 878, 'name': 'Science Fiction'}]
homepage                                                                                                                                                                                                                                                                                                http://www.avatarmovie.com/
id                          

>iloc를 이용하면 index의 정보를 얻을 수 있다. 즉, 영화의 정보를 얻기 위해 iloc에 index 정보를 넣고 그 중 title feature를 출력하여 추천 영화들을 얻을 수 있게 된다.

#7. 영화 추천하기

In [89]:
#영화 이름을 입력 받으면 index 정보 찾기
movie_name = 'avatar'
title_movie = df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]
title_movie

movie_index = title_movie.index.values
movie_index

#index정보를 이용하여 top10
movie_top10 = overview_similarity_sorted_idx[movie_index, :10]
print(type(movie_top10))

movie_top10 = movie_top10.reshape(-1)
print(movie_top10.ndim, '차원 : ', movie_top10)

#top10정보를 이용하여 영화 제목 추출
df_movies.iloc[movie_top10]['title']

<class 'numpy.ndarray'>
1 차원 :  [   0 3603 3191  847  153 4110  420 1517 3949 1471]


0                                     Avatar
3604                               Apollo 18
3192                           Two Evil Eyes
847                                 Semi-Pro
153     Mission: Impossible - Ghost Protocol
4111                             Dinner Rush
420              Hellboy II: The Golden Army
1517                                Repo Men
3950                          Brave New Girl
1471                                   Ponyo
Name: title, dtype: object